In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import time, json, datetime
from tqdm import tqdm
import sys, getopt

import time
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [19]:
class DeepFM(nn.Module):
    def __init__(self, cate_fea_nuniqs, nume_fea_size=0, emb_size=8,
                 hid_dims=[256, 128], num_classes=1, dropout=[0.2, 0.2]):
        """
        cate_fea_nuniqs: 类别特征的唯一值个数列表，也就是每个类别特征的vocab_size所组成的列表
        nume_fea_size: 数值特征的个数，该模型会考虑到输入全为类别型，即没有数值特征的情况
        """
        super().__init__()
        self.cate_fea_size = len(cate_fea_nuniqs)
        self.nume_fea_size = nume_fea_size

        """FM部分"""
        # 一阶
        if self.nume_fea_size != 0:
            self.fm_1st_order_dense = nn.Linear(self.nume_fea_size, 1)  # 数值特征的一阶表示
        self.fm_1st_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, 1) for voc_size in cate_fea_nuniqs])  # 类别特征的一阶表示
        self.fm_1st_order_title = nn.Linear(128, 1)
        
        # 二阶
        self.fm_2nd_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, emb_size) for voc_size in cate_fea_nuniqs])  # 类别特征的二阶表示

        """DNN部分"""
        self.all_dims = [self.cate_fea_size * emb_size] + hid_dims
        self.dense_linear = nn.Linear(self.nume_fea_size, self.cate_fea_size * emb_size)  # 数值特征的维度变换到FM输出维度一致
        self.title_linear = nn.Linear(128, self.cate_fea_size * emb_size)
        self.relu = nn.ReLU()
        # for DNN
        for i in range(1, len(self.all_dims)):
            setattr(self, 'linear_' + str(i), nn.Linear(self.all_dims[i - 1], self.all_dims[i]))
            setattr(self, 'batchNorm_' + str(i), nn.BatchNorm1d(self.all_dims[i]))
            setattr(self, 'activation_' + str(i), nn.ReLU())
            setattr(self, 'dropout_' + str(i), nn.Dropout(dropout[i - 1]))
        # for output
        self.dnn_linear = nn.Linear(hid_dims[-1], num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X_sparse, X_dense, X_title):
        """
        X_sparse: 类别型特征输入  [bs, cate_fea_size]
        X_dense: 数值型特征输入（可能没有）  [bs, dense_fea_size]
        """

        """FM 一阶部分"""
        fm_1st_sparse_res = [emb(X_sparse[:, i].unsqueeze(1)).view(-1, 1)
                             for i, emb in enumerate(self.fm_1st_order_sparse_emb)]
        fm_1st_sparse_res = torch.cat(fm_1st_sparse_res, dim=1)  # [bs, cate_fea_size]
        fm_1st_sparse_res = torch.sum(fm_1st_sparse_res, 1, keepdim=True)  # [bs, 1]
        
        fm_1st_title_res=self.fm_1st_order_title(X_title)
        
        if X_dense is not None:
            fm_1st_dense_res = self.fm_1st_order_dense(X_dense)
            fm_1st_part = fm_1st_sparse_res + fm_1st_dense_res+fm_1st_title_res
        else:
            fm_1st_part = fm_1st_sparse_res+fm_1st_title_res  # [bs, 1]
        """FM 一阶部分"""
        
        """FM 二阶部分"""
        fm_2nd_order_res = [emb(X_sparse[:, i].unsqueeze(1)) for i, emb in enumerate(self.fm_2nd_order_sparse_emb)]
        fm_2nd_concat_1d = torch.cat(fm_2nd_order_res, dim=1)  # [bs, n, emb_size]  n为类别型特征个数(cate_fea_size)

        # 先求和再平方
        sum_embed = torch.sum(fm_2nd_concat_1d, 1)  # [bs, emb_size]
        square_sum_embed = sum_embed * sum_embed  # [bs, emb_size]
        # 先平方再求和
        square_embed = fm_2nd_concat_1d * fm_2nd_concat_1d  # [bs, n, emb_size]
        sum_square_embed = torch.sum(square_embed, 1)  # [bs, emb_size]
        # 相减除以2
        sub = square_sum_embed - sum_square_embed
        sub = sub * 0.5  # [bs, emb_size]

        fm_2nd_part = torch.sum(sub, 1, keepdim=True)  # [bs, 1]
        """FM 二阶部分"""
        
        """DNN部分"""
        dnn_out = torch.flatten(fm_2nd_concat_1d, 1)  # [bs, n * emb_size]
        if X_title is not None:
            title_out=self.relu(self.title_linear(X_title))
            dnn_out= dnn_out+title_out
        
        if X_dense is not None:
            dense_out = self.relu(self.dense_linear(X_dense))  # [bs, n * emb_size]
            dnn_out = dnn_out + dense_out  # [bs, n * emb_size]

        for i in range(1, len(self.all_dims)):
            dnn_out = getattr(self, 'linear_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'batchNorm_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'activation_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'dropout_' + str(i))(dnn_out)

        dnn_out = self.dnn_linear(dnn_out)  # [bs, 1]
        """DNN部分"""
        
        out = fm_1st_part + fm_2nd_part + dnn_out  # [bs, 1]
        out = self.sigmoid(out)
        return out

In [20]:
def train_and_eval(model, train_loader, valid_loader, epochs, optimizer, loss_fcn, scheduler, device,ii):
    import time
    best_auc=0
    for _ in range(epochs):
        """训练部分"""
        model.train()
        print("Current lr : {}".format(optimizer.state_dict()['param_groups'][0]['lr']))
        write_log('Epoch: {}:{}'.format(ii,_))
        train_loss_sum = 0.0
        start_time = time.time()
        for idx, x in enumerate(train_loader):
            cate_fea, nume_fea, title_fea, label = x[0], x[1], x[2] ,x[3]
            cate_fea, nume_fea,title_fea, label = cate_fea.to(device), nume_fea.to(device),title_fea.to(device), label.float().to(device)
            pred = model(cate_fea, nume_fea,title_fea).view(-1)
            loss = loss_fcn(pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.cpu().item()
            if (idx + 1) % 50 == 0 or (idx + 1) == len(train_loader):
                write_log("Epoch {:04d} | Step {:04d} / {} | Loss {:.4f} | Time {:.4f}".format(
                    _ + 1, idx + 1, len(train_loader), train_loss_sum / (idx + 1), time.time() - start_time))
        scheduler.step()
        """推断部分"""
        model.eval()
        with torch.no_grad():
            valid_labels, valid_preds = [], []
            for idx, x in tqdm(enumerate(valid_loader)):
                cate_fea, nume_fea,title_fea, label = x[0], x[1], x[2],x[3]
                cate_fea, nume_fea,title_fea = cate_fea.to(device), nume_fea.to(device),title_fea.to(device)
                pred = model(cate_fea, nume_fea,title_fea).reshape(-1).data.cpu().numpy().tolist()
                valid_preds.extend(pred)
                valid_labels.extend(label.cpu().numpy().tolist())
        cur_auc = roc_auc_score(valid_labels, valid_preds)
        if cur_auc > best_auc:
            best_auc = cur_auc
            import time
            end=time.time()
            torch.save(model.state_dict(), "../../data/wj/deepfm_best/fen10 3.0/deepfm_best10fen_title_"+str(round(best_auc,4))+"_"+str(ii)+"_"+str(_)+"_"+str(time.strftime('%m_%d_%H_%M_%S'))+".pth")
        write_log('Current AUC: %.6f, Best AUC: %.6f\n' % (cur_auc, best_auc))




# 定义日志（data文件夹下，同级目录新建一个data文件夹）
def write_log(w):
    file_name = '../../data/wj/' + datetime.date.today().strftime('%m%d') + "_{}.log".format("deepfm_10fen")
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info + '\n')
        
        

In [21]:
doc_feat = pd.read_pickle('../../data/wj/doc.pkl')
user_feat = pd.read_pickle('../../data/wj/user.pkl')
df_test_user_doc=pd.read_pickle('../../data/wj/df_test_user_doc_64.pkl')

sparse_features = ['userid', 'docid', 'network', 'hour', 'device', 'os', 'province',
                   'city', 'age', 'gender', 'category1st', 'category2nd',
                   'pub_date', 'keyword0', 'keyword1', 'keyword2', 'keyword3', 'keyword4']

dense_features = ['refresh', 'picnum',
                  'userid_click_mean','userid_click_count' ,'userid_duration_mean' ,'userid_picnum_mean',
                    'docid_click_mean','docid_click_count','docid_duration_mean','docid_picnum_mean',
                    'category1st_click_mean','category1st_click_count','category1st_duration_mean','category1st_picnum_mean',
                    'category2nd_click_mean','category2nd_click_count','category2nd_duration_mean','category2nd_picnum_mean',
                    'keyword0_click_mean','keyword0_click_count','keyword0_duration_mean','keyword0_picnum_mean',
                 'network_click_mean', 'network_click_count', 'network_duration_mean', 
                  'refresh_click_mean', 'refresh_click_count', 'refresh_duration_mean',
                  'device_click_mean', 'device_click_count', 'device_duration_mean', 
                  'os_click_mean', 'os_click_count', 'os_duration_mean', 
                  'province_click_mean', 'province_click_count', 'province_duration_mean', 
                  'city_click_mean', 'city_click_count', 'city_duration_mean', 
                  'age_click_mean', 'age_click_count', 'age_duration_mean', 
                  'gender_click_mean', 'gender_click_count', 'gender_duration_mean'
                 ]

cate_fea_nuniqs = []
cate_fea_nuniqs.append(user_feat['userid'].nunique() + 1)
cate_fea_nuniqs.append(doc_feat['docid'].nunique() + 1)
cate_fea_nuniqs.append(6)  # network
cate_fea_nuniqs.append(13)  # hour
cate_fea_nuniqs.append(user_feat['device'].nunique())
cate_fea_nuniqs.append(user_feat['os'].nunique())
cate_fea_nuniqs.append(user_feat['province'].nunique())
cate_fea_nuniqs.append(user_feat['city'].nunique())
cate_fea_nuniqs.append(user_feat['age'].nunique())
cate_fea_nuniqs.append(user_feat['gender'].nunique())
cate_fea_nuniqs.append(doc_feat['category1st'].nunique())
cate_fea_nuniqs.append(doc_feat['category2nd'].nunique())
cate_fea_nuniqs.append(doc_feat['pub_date'].nunique())
keyword_nunique = max(doc_feat['keyword0'].max(), doc_feat['keyword1'].max(), doc_feat['keyword2'].max()
                      , doc_feat['keyword3'].max(), doc_feat['keyword4'].max()) + 1
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)

device = torch.device('cuda:2') if torch.cuda.is_available() else torch.device('cpu')
loss_fcn = nn.BCELoss()
loss_fcn = loss_fcn.to(device)

In [22]:
import pandas as pd
title_=pd.read_pickle('../../data/wj/title_embedding.pkl')
del title_['title']

In [ ]:
import pandas as pd


len_=81671133
ii=-1
for i in range(0,len_,round(len_/10)):
    ii+=1
    right=i+round(len_/10)
    if right>=len_:
        break
    print(i,right)
    df_train_user_doc=pd.read_pickle('../../data/wj/df_train_user_doc_0_1_'+str(i)+'_'+str(right)+'_64.pkl')
    df_train_user_doc=pd.merge(df_train_user_doc,title_,how='left',on='docid')
    
    train, valid = train_test_split(df_train_user_doc, test_size=0.25, random_state=2021)
#         train_loader，valid_loader
    train_dataset = Data.TensorDataset(torch.LongTensor(train[sparse_features].values),
                                           torch.FloatTensor(train[dense_features].values),
                                           torch.FloatTensor(np.stack(train['title_'].values,axis=0)),
                                           torch.FloatTensor(train['click'].values), )
    train_loader = Data.DataLoader(dataset=train_dataset, batch_size=8192, shuffle=True)
    valid_dataset = Data.TensorDataset(torch.LongTensor(valid[sparse_features].values),
                                           torch.FloatTensor(valid[dense_features].values),
                                           torch.FloatTensor(np.stack(valid['title_'].values,axis=0)),
                                           torch.FloatTensor(valid['click'].values))
    valid_loader = Data.DataLoader(dataset=valid_dataset, batch_size=8192, shuffle=False)
    
#         train
    print('train')
    model = DeepFM(cate_fea_nuniqs, nume_fea_size=len(dense_features))
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
    epoch_=10
    train_and_eval(model, train_loader, valid_loader, epoch_, optimizer, loss_fcn, scheduler, device,ii)

0 8167113
train
Current lr : 0.005
09:27:01 : Epoch: 0:0
09:27:22 : Epoch 0001 | Step 0050 / 748 | Loss 11.9431 | Time 21.1444
09:27:41 : Epoch 0001 | Step 0100 / 748 | Loss 8.5201 | Time 39.7328
09:27:59 : Epoch 0001 | Step 0150 / 748 | Loss 6.4106 | Time 57.7807
09:28:17 : Epoch 0001 | Step 0200 / 748 | Loss 5.0532 | Time 76.4202
09:28:36 : Epoch 0001 | Step 0250 / 748 | Loss 4.1820 | Time 94.5529
09:28:53 : Epoch 0001 | Step 0300 / 748 | Loss 3.5776 | Time 111.9283
09:29:10 : Epoch 0001 | Step 0350 / 748 | Loss 3.1349 | Time 129.3362
09:29:28 : Epoch 0001 | Step 0400 / 748 | Loss 2.7975 | Time 147.2616
09:29:46 : Epoch 0001 | Step 0450 / 748 | Loss 2.5319 | Time 165.1027
09:30:04 : Epoch 0001 | Step 0500 / 748 | Loss 2.3177 | Time 183.1542
09:30:22 : Epoch 0001 | Step 0550 / 748 | Loss 2.1413 | Time 201.1811
09:30:40 : Epoch 0001 | Step 0600 / 748 | Loss 1.9937 | Time 218.9442
09:30:57 : Epoch 0001 | Step 0650 / 748 | Loss 1.8685 | Time 236.4289
09:31:15 : Epoch 0001 | Step 0700 / 7

250it [01:14,  3.35it/s]


09:32:49 : Current AUC: 0.760207, Best AUC: 0.760207

Current lr : 0.004
09:32:49 : Epoch: 0:1
09:33:11 : Epoch 0002 | Step 0050 / 748 | Loss 0.3561 | Time 21.3875
09:33:31 : Epoch 0002 | Step 0100 / 748 | Loss 0.3555 | Time 41.6168
09:33:50 : Epoch 0002 | Step 0150 / 748 | Loss 0.3555 | Time 61.0931
09:34:10 : Epoch 0002 | Step 0200 / 748 | Loss 0.3553 | Time 80.4266
09:34:29 : Epoch 0002 | Step 0250 / 748 | Loss 0.3548 | Time 100.2456
09:34:49 : Epoch 0002 | Step 0300 / 748 | Loss 0.3545 | Time 119.5732
09:35:07 : Epoch 0002 | Step 0350 / 748 | Loss 0.3543 | Time 138.1268
09:35:27 : Epoch 0002 | Step 0400 / 748 | Loss 0.3545 | Time 158.0096
09:35:47 : Epoch 0002 | Step 0450 / 748 | Loss 0.3543 | Time 177.8406
09:36:07 : Epoch 0002 | Step 0500 / 748 | Loss 0.3542 | Time 197.6384
09:36:26 : Epoch 0002 | Step 0550 / 748 | Loss 0.3540 | Time 217.0410
09:36:45 : Epoch 0002 | Step 0600 / 748 | Loss 0.3540 | Time 235.7293
09:37:04 : Epoch 0002 | Step 0650 / 748 | Loss 0.3539 | Time 254.8986

250it [01:13,  3.40it/s]


09:38:56 : Current AUC: 0.770861, Best AUC: 0.770861

Current lr : 0.0032
09:38:56 : Epoch: 0:2
09:39:17 : Epoch 0003 | Step 0050 / 748 | Loss 0.3513 | Time 21.4117
09:39:37 : Epoch 0003 | Step 0100 / 748 | Loss 0.3507 | Time 40.6053
09:39:56 : Epoch 0003 | Step 0150 / 748 | Loss 0.3518 | Time 59.9693
09:40:16 : Epoch 0003 | Step 0200 / 748 | Loss 0.3511 | Time 79.8630
09:40:37 : Epoch 0003 | Step 0250 / 748 | Loss 0.3506 | Time 100.9327
09:40:57 : Epoch 0003 | Step 0300 / 748 | Loss 0.3508 | Time 120.6489
09:41:17 : Epoch 0003 | Step 0350 / 748 | Loss 0.3509 | Time 141.0619
09:41:38 : Epoch 0003 | Step 0400 / 748 | Loss 0.3510 | Time 161.6528
09:41:57 : Epoch 0003 | Step 0450 / 748 | Loss 0.3510 | Time 181.3358
09:42:17 : Epoch 0003 | Step 0500 / 748 | Loss 0.3511 | Time 200.7855
09:42:36 : Epoch 0003 | Step 0550 / 748 | Loss 0.3512 | Time 219.7763
09:42:55 : Epoch 0003 | Step 0600 / 748 | Loss 0.3511 | Time 238.8442
09:43:15 : Epoch 0003 | Step 0650 / 748 | Loss 0.3510 | Time 259.404

...

10:22:28 : Current AUC: 0.774244, Best AUC: 0.775880

Current lr : 0.0006710886400000004
10:22:28 : Epoch: 0:9
10:22:48 : Epoch 0010 | Step 0050 / 748 | Loss 0.3466 | Time 20.4464
10:23:08 : Epoch 0010 | Step 0100 / 748 | Loss 0.3479 | Time 39.7958
10:23:27 : Epoch 0010 | Step 0150 / 748 | Loss 0.3478 | Time 59.4082
10:23:47 : Epoch 0010 | Step 0200 / 748 | Loss 0.3482 | Time 78.7784
10:24:06 : Epoch 0010 | Step 0250 / 748 | Loss 0.3486 | Time 98.3309
10:24:26 : Epoch 0010 | Step 0300 / 748 | Loss 0.3486 | Time 117.8223
10:24:45 : Epoch 0010 | Step 0350 / 748 | Loss 0.3486 | Time 137.3632
10:25:05 : Epoch 0010 | Step 0400 / 748 | Loss 0.3485 | Time 156.9472
10:25:25 : Epoch 0010 | Step 0450 / 748 | Loss 0.3487 | Time 176.5433
10:25:45 : Epoch 0010 | Step 0500 / 748 | Loss 0.3487 | Time 196.5601
10:26:04 : Epoch 0010 | Step 0550 / 748 | Loss 0.3487 | Time 216.1945
10:26:24 : Epoch 0010 | Step 0600 / 748 | Loss 0.3488 | Time 236.1412
10:26:44 : Epoch 0010 | Step 0650 / 748 | Loss 0.3488 

250it [01:17,  3.24it/s]


10:28:42 : Current AUC: 0.775772, Best AUC: 0.775880

8167113 16334226
train
Current lr : 0.005
10:29:35 : Epoch: 1:0
10:29:57 : Epoch 0001 | Step 0050 / 748 | Loss 11.2514 | Time 21.6037
10:30:16 : Epoch 0001 | Step 0100 / 748 | Loss 7.9494 | Time 40.5666
10:30:35 : Epoch 0001 | Step 0150 / 748 | Loss 5.8798 | Time 60.0191
10:30:54 : Epoch 0001 | Step 0200 / 748 | Loss 4.6397 | Time 78.9971
10:31:13 : Epoch 0001 | Step 0250 / 748 | Loss 3.8463 | Time 98.0520
10:31:33 : Epoch 0001 | Step 0300 / 748 | Loss 3.2952 | Time 117.7266
10:31:52 : Epoch 0001 | Step 0350 / 748 | Loss 2.8917 | Time 136.8876
10:32:11 : Epoch 0001 | Step 0400 / 748 | Loss 2.5837 | Time 155.8119
10:32:29 : Epoch 0001 | Step 0450 / 748 | Loss 2.3413 | Time 174.0677
10:32:48 : Epoch 0001 | Step 0500 / 748 | Loss 2.1457 | Time 192.5338
10:33:06 : Epoch 0001 | Step 0550 / 748 | Loss 1.9850 | Time 211.2458
10:33:24 : Epoch 0001 | Step 0600 / 748 | Loss 1.8505 | Time 229.3189
10:33:43 : Epoch 0001 | Step 0650 / 748 | Loss

250it [01:19,  3.13it/s]


10:35:42 : Current AUC: 0.758112, Best AUC: 0.758112

Current lr : 0.004
10:35:42 : Epoch: 1:1


In [38]:
torch.save(model.state_dict(), "../../data/wj/deepfm_best_"+str(0.7791)+"_"+str(1)+"_"+str(9)+".pth")

In [22]:
# df_train_user_doc.to_pickle('../../data/wj/df_train_user_doc_0.1.pkl')

In [5]:
# model = DeepFM(cate_fea_nuniqs, nume_fea_size=len(dense_features))
# model.load_state_dict(torch.load('../../data/wj/deepfm_best_0.7739082162724398_1639060823.4660978.pth'))
# model.to(device)
# epoch = 5
# train_and_eval(model, train_loader, valid_loader, epoch, optimizer, loss_fcn, scheduler, device)